In [1]:
import datetime
import sys

sys.path.append("..")

from src.auth.aurora import get_portal_client, get_sales_client

In [2]:
portal_client = get_portal_client()
sales_client = get_sales_client()

In [3]:
client_id = "03dd22e9-4203-11ec-be49-061393e0f1a7"

In [4]:
device_serials = sales_client.query_to_list(
    """SELECT DISTINCT device_id
    FROM sales.t_subscription AS s
    INNER JOIN sales.t_orders AS o
    USING(order_number)
    LEFT JOIN sales.t_cancellation AS c
    USING(subscription_id)
    WHERE client_id = %(client_id)s
    AND item_code = "101-104-02"
    AND c.cancel_date IS NULL
    AND s.expiry_date > current_date()""",
    {"client_id": client_id},
)

In [5]:
len(device_serials)

68

In [6]:
placeholders = ', '.join(['%s'] * len(device_serials))
go_usage_df = portal_client.query_to_pandas(
    f"""SELECT
        d.id,
        d.device_serial,
        d.name,
        d.created_at,
        greatest(MAX(d_c.ended_at), MAX(d_c.created_at)) AS last_connected
    FROM topaz.api_devices AS d
    LEFT JOIN topaz.api_device_connections AS d_c
    ON d_c.device_id = d.id
    WHERE d.device_type_id = 14
    AND d.device_serial IN ({placeholders})
    AND d.deleted_at IS NULL
    GROUP BY d.id, d.name""",
    params=device_serials
)

In [7]:
go_usage_df.sort_values("last_connected").to_csv("data/2025-07-29 Recent Visits for Marlowe.csv")

In [8]:
go_usage_df

,id,device_serial,name,created_at,last_connected
0,22206,G01-00941,Adam Prendergast (G01-00941),2024-11-09 15:10:51,2025-07-16 13:31:47
1,23354,G01-01107,Dom Wharton (G01-01107),2025-01-26 09:18:11,2025-07-24 15:01:07
2,25636,G01-01308,Dale Windmill (G01-01308),2025-05-31 09:02:22,2025-07-10 07:42:25
3,25637,G01-01309,Frank Andrady (G01-01309),2025-05-31 09:03:47,2025-07-10 11:00:16
4,25638,G01-01310,Harley Page (G01-01310),2025-05-31 09:36:38,2025-07-30 10:19:38
...,...,...,...,...,...
63,26949,G01-01454,None,2025-07-26 18:57:31,2025-07-26 19:03:37
64,26950,G01-01455,None,2025-07-26 19:00:28,2025-07-26 19:07:07
65,26951,G01-01456,None,2025-07-26 19:02:28,2025-07-26 19:09:28
66,26952,G01-01457,None,2025-07-26 19:06:03,2025-07-26 19:11:56
